In [4]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('DeepLearningMovies/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np

In [142]:
import os

txt1 = "i characters characters really fucking love that movie. I enjoyed the charaters play and so on. looking forward a movie like that .... "
txt2 = "It was so annoying. i hate that type of movie :("
rootdirPOS ="./data/train/pos"
rootdirNEG ="./data/train/neg"
pos = os.listdir("./data/train/pos")
neg = os.listdir("./data/train/neg")
train = []
for i in range(100):
    with open("./data/train/pos/"+pos[i], 'r') as content_file:
        content = content_file.read();
        train.append(content)
    with open("./data/train/neg/"+neg[i], 'r') as content_file:
        content = content_file.read();
        train.append(content)
y_train=np.ones(len(train))
y_train[y_train.shape[0]/2:]=0

test = []
fil = os.listdir("./data/test/")
for i in range(200):
    with open("./data/test/"+fil[i], 'r') as content_file:
        content = content_file.read();
        test.append(content)


In [186]:
clean_train

['felix unger jack lemmon has just been dumped by his wife because he is one of the most annoying neurotic people in the world suicide is his way out but he just can t seem to get it to work so he heads over to his friends house oscar madison walther matthau is also recently divorced and living it up in bachelor heaven smoking gambling hitting on chicks eating out and never cleaning is paradise to him well with the suicide attempts oscar decides to let felix move in at first it is a match made in heaven felix cooks and cleans and helps oscar pay his alimony on time but soon oscar is jonesing for women and felix who in today s world would probably be gay isn t ready to move on they invite a couple of british birds over and they find felix so tender that soon they and felix are weeping and chatting about his family life leaving oscar denied this is it he explodes and throws him out but felix isn t as helpless as it seems and soon he has the upper hand my favorite quote you leave me littl

In [205]:
import re
import string
import nltk
from nltk.stem.porter import *
from nltk.util import ngrams
from nltk.corpus import stopwords

def doc2words(doc,remove_stopwords=False):
	doc = re.sub("[^a-zA-Z]"," ", doc)
	#doc = re.sub("[^0-9a-zA-Z]"," ", doc)
	# Split in words
	words=doc.strip().split()
	words = [w.lower() for w in words]
	# stop-words based on the previous lab we add nltk stopwords
	if remove_stopwords:
		stop_words = ['review','reviews','could','would','films','film',
					'movies','movie','br','http','also','seem','comment',
					'comments','user','users','imdb','movies', ' ', '']
		stop_words.extend([w.encode('ascii','ignore') for w in stopwords.words("english")])
		stop_words.extend([w for w in words if len(w)<3]) 
		stop_words = set(stop_words)
		words = [w for w in words if w not in stop_words]
	return words

clean_train = [" ".join(doc2words(doc)) for doc in train]
clean_test = [" ".join(doc2words(doc)) for doc in test]
clean_train[0]

'felix unger jack lemmon has just been dumped by his wife because he is one of the most annoying neurotic people in the world suicide is his way out but he just can t seem to get it to work so he heads over to his friends house oscar madison walther matthau is also recently divorced and living it up in bachelor heaven smoking gambling hitting on chicks eating out and never cleaning is paradise to him well with the suicide attempts oscar decides to let felix move in at first it is a match made in heaven felix cooks and cleans and helps oscar pay his alimony on time but soon oscar is jonesing for women and felix who in today s world would probably be gay isn t ready to move on they invite a couple of british birds over and they find felix so tender that soon they and felix are weeping and chatting about his family life leaving oscar denied this is it he explodes and throws him out but felix isn t as helpless as it seems and soon he has the upper hand my favorite quote you leave me little

In [174]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( doc2words( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [175]:
sentences = []  # Initialize an empty list of sentences
import nltk.data
#nltk.download()   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print "Parsing sentences from training set"
for review in train:
    sentences += review_to_sentences(unicode(review,errors='ignore'), tokenizer)


Parsing sentences from training set


In [176]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)


Training model...


In [209]:
stop_words = ['review','reviews','could','would','films','film',
					'movies','movie','br','http','also','seem','comment',
					'comments','user','users','imdb','movies', ' ', '']
stop_words.extend([w.encode('ascii','ignore') for w in stopwords.words("english")])
set(stop_words)

{'',
 ' ',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'also',
 'am',
 'an',
 'and',
 'any',
 'are',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'br',
 'but',
 'by',
 'can',
 'comment',
 'comments',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'don',
 'down',
 'during',
 'each',
 'few',
 'film',
 'films',
 'for',
 'from',
 'further',
 'had',
 'has',
 'have',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'http',
 'i',
 'if',
 'imdb',
 'in',
 'into',
 'is',
 'it',
 'its',
 'itself',
 'just',
 'me',
 'more',
 'most',
 'movie',
 'movies',
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'now',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'review',
 'reviews',
 's',
 'same',
 'seem',
 'she',
 'should',
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None)
train_feautures = vectorizer.fit_transform(clean_train)
train_feautures = train_feautures.toarray()

In [51]:
clean_train[0]


'felix unger jack lemmon has just been dumped by his wife because he is one of the most annoying neurotic people in the world suicide is his way out but he just can t seem to get it to work so he heads over to his friends house oscar madison walther matthau is also recently divorced and living it up in bachelor heaven smoking gambling hitting on chicks eating out and never cleaning is paradise to him well with the suicide attempts oscar decides to let felix move in at first it is a match made in heaven felix cooks and cleans and helps oscar pay his alimony on time but soon oscar is jonesing for women and felix who in today s world would probably be gay isn t ready to move on they invite a couple of british birds over and they find felix so tender that soon they and felix are weeping and chatting about his family life leaving oscar denied this is it he explodes and throws him out but felix isn t as helpless as it seems and soon he has the upper hand my favorite quote you leave me little

In [57]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
if __name__ == '__main__':
    train = pd.read_csv(os.path.join(os.path.dirname(__file__), 'data', 'labeledTrainData.tsv'), header=0, \
                    delimiter="\t", quoting=3)
    test = pd.read_csv(os.path.join(os.path.dirname(__file__), 'data', 'testData.tsv'), header=0, delimiter="\t", \
                   quoting=3 )

    print 'The first review is:'
    print train["review"][0]

    raw_input("Press Enter to continue...")


    print 'Download text data sets. If you already have NLTK datasets downloaded, just close the Python download window...'
    #nltk.download()  # Download text data sets, including stop words

    # Initialize an empty list to hold the clean reviews
    clean_train_reviews = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the movie review list

    print "Cleaning and parsing the training set movie reviews...\n"
    for i in xrange( 0, len(train["review"])):
        clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))


    # ****** Create a bag of words from the training set
    #
    print "Creating the bag of words...\n"


    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.
    vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an
    # array
    train_data_features = train_data_features.toarray()

    # ******* Train a random forest using the bag of words
    #
    print "Training the random forest (this may take a while)..."


    # Initialize a Random Forest classifier with 100 trees
    forest = RandomForestClassifier(n_estimators = 100)

    # Fit the forest to the training set, using the bag of words as
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit( train_data_features, train["sentiment"] )



    # Create an empty list and append the clean reviews one by one
    clean_test_reviews = []

    print "Cleaning and parsing the test set movie reviews...\n"
    for i in xrange(0,len(test["review"])):
        clean_test_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True)))

    # Get a bag of words for the test set, and convert to a numpy array
    test_data_features = vectorizer.transform(clean_test_reviews)
    test_data_features = test_data_features.toarray()

    # Use the random forest to make sentiment label predictions
    print "Predicting test labels...\n"
    result = forest.predict(test_data_features)

    # Copy the results to a pandas dataframe with an "id" column and
    # a "sentiment" column
    output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

    # Use pandas to write the comma-separated output file
    output.to_csv(os.path.join(os.path.dirname(__file__), 'data', 'Bag_of_Words_model.csv'), index=False, quoting=3)
    print "Wrote results to Bag_of_Words_model.csv"




NameError: name '__file__' is not defined